In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import torchvision
from torchvision.io import read_image
from random import  uniform


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import InterpolationMode

In [27]:
path  = "../input/face-mask-12k-images-dataset/Face Mask Dataset/"

In [28]:
dataset = {"image_path":[],"mask_status":[],"where":[]}
for where in os.listdir(path):
    for status in os.listdir(path+"/"+where):
        for image in glob.glob(path+where+"/"+status+"/"+"*.png"):
            dataset["image_path"].append(image)
            dataset["mask_status"].append(status)
            dataset["where"].append(where)
dataset = pd.DataFrame(dataset)

In [29]:
dataset.mask_status=dataset.mask_status.map({ 'WithMask' : 1, 'WithoutMask' : 0})

In [30]:
dataset

In [31]:
read_image(dataset['image_path'][87]).shape

In [32]:
print(f"With Mask: {len(dataset[dataset['mask_status']==1])},\nWithout Mask: {len(dataset[dataset['mask_status']==0])}\n")
sns.countplot(dataset["mask_status"])
plt.show()

In [33]:
plt.figure(figsize = (14,10))
for i in range(9):
    random = np.random.randint(1,len(dataset))
    plt.subplot(3,3,i+1)
    plt.imshow(cv2.imread(dataset.loc[random,"image_path"]))
    plt.title(dataset.loc[random, "mask_status"], size = 10, color = "purple") 
    plt.xticks([])
    plt.yticks([])
    
plt.show()

# PreProcessing Images

In [34]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(100,100),interpolation=InterpolationMode.BILINEAR),
    #torchvision.transforms.CenterCrop(size=(200,200)),
    torchvision.transforms.ColorJitter(brightness=uniform(0,1), contrast=uniform(0,1), saturation=uniform(0,1), hue=uniform(0,0.5)),
    #torchvision.transforms.RandomCrop((200, 200), fill=(0,0,0), padding_mode='edge'),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomPerspective(distortion_scale=np.random.rand(), p=0.7, fill=0),
    torchvision.transforms.RandomRotation(degrees = int(uniform(0,180)))
])

# Data Loader

In [35]:
class CustomImageDataset(Dataset):
    def __init__(self,dataset,transform =transforms, target_transform = None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,idx):
        image = read_image(self.dataset['image_path'].iloc[idx])
        label = self.dataset['mask_status'].iloc[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [36]:
train_loader = data_utils.DataLoader(CustomImageDataset(dataset[dataset['where'] == 'Train']), batch_size=64, shuffle=True)
test_loader = data_utils.DataLoader(CustomImageDataset(dataset[dataset['where'] == 'Test']), batch_size=64, shuffle=True)
val_loader = data_utils.DataLoader(CustomImageDataset(dataset[dataset['where'] == 'Validation']), batch_size=64, shuffle=True)

# Model

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [38]:
class nNet(nn.Module):
    def __init__(self):
        super(nNet,self).__init__()
        self.conv1 = nn.Conv2d(3,32,kernel_size = 1, padding = 'same',stride =1 )
        self.conv2 = nn.Conv2d(32,64,kernel_size = 1, padding = 'same',stride =1 )
        self.conv3 = nn.Conv2d(64,128,kernel_size = 1, padding = 'same',stride =1 )
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c_drop = nn.Dropout2d()
        
        self.fc1 = nn.Linear(25*25*128,64)
        self.fc2 = nn.Linear(64,2)
        
    def forward(self,x):
        #print('input '+str(x.shape))
        no_samples = x.shape[0]
        x=F.relu(self.conv1(x))
        #print('conv1 '+str(x.shape))
        x=self.max_pool2d(x)
        #print('maxpool1 '+str(x.shape))
        x=F.relu(self.conv2(x))
        #print('conv2 '+str(x.shape))
        x=self.max_pool2d(x)
        #print('maxpool2 '+str(x.shape))
        x = F.relu(self.conv3(x))
        #print('conv3 '+str(x.shape))
        x=x.view(64,-1)
        #print('view '+str(x.shape))
        x = F.relu(self.fc1(x))
        #print('fc1 '+str(x.shape))
        x = F.dropout(x, training=self.training)
        #print('dropout '+str(x.shape))
        x = F.relu(self.fc2(x))
        #print('fc2 '+str(x.shape))
        return F.log_softmax(x,dim=1)

In [39]:
model = nNet()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.05)
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

In [40]:
model

In [41]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(4)]

In [42]:
log_interval = 10
def train_fn(epoch):
    model.train()
    for batch_idx, (data,targets) in enumerate(train_loader):
        if len(targets)== 64:
            optimizer.zero_grad()
            output = model(data.float())
            loss=F.nll_loss(output,targets)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))
                train_losses.append(loss.item())
                train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
                #torch.save(network.state_dict(), '/results/model.pth')
                #torch.save(optimizer.state_dict(), '/results/optimizer.pth')
        else:
            continue

In [43]:
def test_fn():
    print('test')
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if len(target) == 64:
                output = model(data.float())
                #print("output    "+ str(output.shape))
                #print(output.dtype)
                #print("target     "+ str(target.shape))
                test_loss += F.nll_loss(output, target, size_average=False).item()
                pred = output.data.max(1, keepdim=True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()
            else:
                continue
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))

In [44]:
test_fn()
for epoch in range(1, 2):
    train_fn(epoch)
    test_fn()